In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 96 kB 4.1 MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-py3-none-any.whl size=9568 sha256=0781003bdf4dd34552bbc5387ea386c8bfd86077d76e1779b4fa893cdaf1aedb
  Stored in directory: /root/.cache/pip/wheels/38/5c/e0/13003e68c17f403af40b92a24d20171b95fef13b0fdaba833c
Successfully built kt-legacy


In [50]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [51]:
fashion_mnist = keras.datasets.fashion_mnist

In [52]:
(train_images, train_labels) , (test_images, test_labels) = fashion_mnist.load_data()

In [53]:
train_images = train_images/255.0
test_images = test_images/255.0

In [54]:
train_images[0].shape

(28, 28)

In [55]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [56]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters = hp.Int('conv_1_filter', min_value = 64, max_value = 128, step=16),
          kernel_size = hp.Choice('conv_1_kernel', values = [3,5]),
          activation = 'relu',
          input_shape=(28, 28, 1)          
      ),
      keras.layers.Conv2D(
          filters = hp.Int('conv_2_filter', min_value = 32, max_value = 64, step=16),
          kernel_size = hp.Choice('conv_2_kernel', values = [3,5]),
          activation = 'relu',          
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step=16),
          activation='relu'
      ),
      keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

In [57]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [58]:
tuner_search = RandomSearch(build_model, objective = 'val_accuracy', max_trials=5, directory='output', project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [59]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [60]:
model = tuner_search.get_best_models(num_models=1)[0]

In [61]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        96048     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2602096   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,700,074
Trainable params: 2,700,074
Non-trainable params: 0
_________________________________________________________________


In [62]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1866 - accuracy: 0.9312 - val_loss: 0.2506 - val_accuracy: 0.9088
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1385 - accuracy: 0.9475 - val_loss: 0.2535 - val_accuracy: 0.9140
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1009 - accuracy: 0.9619 - val_loss: 0.2901 - val_accuracy: 0.9155
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0715 - accuracy: 0.9741 - val_loss: 0.3698 - val_accuracy: 0.9050
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0508 - accuracy: 0.9815 - val_loss: 0.4204 - val_accuracy: 0.9093
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0383 - accuracy: 0.9856 - val_loss: 0.4654 - val_accuracy: 0.9067
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0339 - accuracy: 0.9880 - val_loss: 0.4524 - val_accuracy